<a href="https://colab.research.google.com/github/HarshithMuramalla/HarshithMuramalla/blob/main/Conversational_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets requests torch

In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.cuda.empty_cache()

model_name = "microsoft/DialoGPT-medium"
#model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')  # Move to GPU

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [33]:
import requests

def fetch_trending_tech_news(api_key):
    url = f'https://newsapi.org/v2/top-headlines?category=technology&apiKey={api_key}'  # Use the variable api_key
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Error fetching news: {response.status_code} - {response.text}")
        return []

    data = response.json()
    return data['articles'] if 'articles' in data else []

In [34]:
api_key = 'a784c51d51cd44f3b7913cb5f7aca78e'  # Your actual API key
articles = fetch_trending_tech_news(api_key)

# Check the number of articles fetched
print(f"Number of articles fetched: {len(articles)}")

# Print the first few articles to verify
for article in articles[:5]:  # Print the first 5 articles
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}\n")


Number of articles fetched: 20
Title: Nintendo Expands Switch Online's SNES Library With Four More Titles - Nintendo Life
Description: Arriving next week

Title: 30 Hidden Gems On Amazon That You've Probably Never Heard Of - BuzzFeed
Description: None

Title: Gemini app rolling out Audio Overviews to Android, iOS - 9to5Google
Description: The Gemini app is getting the ability to generate Audio Overview podcasts, with Android and iOS users beginning to see this capability.

Title: Apple faces lawsuit over Apple Intelligence delays - TechCrunch
Description: Apple has been sued in federal court over what plaintiffs allege is false advertising of several Apple Intelligence features.

Title: Alan Wake 2 Devs Designed Their Multiplayer Shooter For People With Kids And Jobs - Kotaku
Description: Remedy Entertainment's FBC: Firebreak won't have seasons, but will have paid cosmetics



In [35]:
from datasets import Dataset
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Set a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as the padding token

# Create the data dictionary from articles
data = {
    'input_column': [article['title'] for article in articles if article.get('title') and article.get('description')],
    'target_column': [article['description'] for article in articles if article.get('title') and article.get('description')]
}

# Check the data structure
print(f"Input Column: {data['input_column'][:5]}")  # Print first 5 titles
print(f"Target Column: {data['target_column'][:5]}")  # Print first 5 descriptions

# Create the dataset
dataset = Dataset.from_dict(data)

def preprocess_function(examples):
    # Tokenize the inputs and targets
    model_inputs = tokenizer(examples['input_column'], max_length=512, truncation=True, padding='max_length')

    # Set the labels for the model
    labels = tokenizer(examples['target_column'], max_length=512, truncation=True, padding='max_length')['input_ids']
    model_inputs["labels"] = labels

    return model_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Check the number of samples in the tokenized dataset
num_samples = len(tokenized_dataset)
print(f"Number of samples in the tokenized dataset: {num_samples}")

Input Column: ["Nintendo Expands Switch Online's SNES Library With Four More Titles - Nintendo Life", 'Gemini app rolling out Audio Overviews to Android, iOS - 9to5Google', 'Apple faces lawsuit over Apple Intelligence delays - TechCrunch', 'Alan Wake 2 Devs Designed Their Multiplayer Shooter For People With Kids And Jobs - Kotaku', 'Painkiller reimagining announced for PS5, Xbox Series, and PC - Gematsu']
Target Column: ['Arriving next week', 'The Gemini app is getting the ability to generate Audio Overview podcasts, with Android and iOS users beginning to see this capability.', 'Apple has been sued in federal court over what plaintiffs allege is false advertising of several Apple Intelligence features.', "Remedy Entertainment's FBC: Firebreak won't have seasons, but will have paid cosmetics", 'Publisher 3D Realms and developer Anshar Studios have announced Painkiller, a reimagining of the classic first-person shooter franchise, for PlayStation 5, Xbox Series, and PC (Steam). It will l

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Number of samples in the tokenized dataset: 19


In [36]:
import os
import torch
from transformers import Trainer, TrainingArguments

# Set environment variable for PyTorch CUDA memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# Clear GPU memory before training
torch.cuda.empty_cache()

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Keep this low
    per_device_eval_batch_size=1,    # Keep this low
    num_train_epochs=3,
    weight_decay=0.01,
    remove_unused_columns=False,
    gradient_accumulation_steps=4,    # Reduce this if needed
    fp16=True,                         # Enable mixed precision
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train the model
trainer.train()





/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=12, training_loss=2.713935216267904, metrics={'train_runtime': 50.7537, 'train_samples_per_second': 1.123, 'train_steps_per_second': 0.236, 'total_flos': 42720231948288.0, 'train_loss': 2.713935216267904, 'epoch': 2.4210526315789473})

In [ ]:
import os
import torch
import requests
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the DialoGPT-medium model and tokenizer
model_name = "microsoft/DialoGPT-medium"  # Use DialoGPT-medium
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')  # Move to GPU

# Function to fetch trending technology news
def fetch_trending_tech_news(api_key):
    url = f'https://newsapi.org/v2/top-headlines?category=technology&apiKey={api_key}'
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Error fetching news: {response.status_code} - {response.text}")
        return []

    data = response.json()
    return data['articles'] if 'articles' in data else []

# Chat function
def chat_with_model(api_key):
    print("Chat with Kozaan! Type 'exit' to end the conversation.")
    print("Type 'news' to get the latest technology news.")

    chat_history_ids = None  # Initialize chat history for DialoGPT

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'exit':
            print("Kozaan: Goodbye! Have a great day!")
            break

        if user_input.lower() == 'what is the latest news':
            articles = fetch_trending_tech_news(api_key)
            if articles:
                print("Kozaan: Here is the latest technology news:")
                for article in articles[:5]:  # Print the first 5 articles
                    print(f"Title: {article['title']}")
                    print(f"Description: {article['description']}\n")
            else:
                print("Kozaan: I couldn't find any news articles at the moment.")
            continue  # Skip to the next iteration
        prompt = (
            "The following is a conversation with Kozan, an AI assistant. "
            "Kozan is helpful, friendly, and always refers to itself as Kozan.\n"
            f"User: {user_input}\nKozan:"
        )

        # Encode the user input and append to chat history
        new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt').to('cuda')

        # Concatenate new user input with chat history (if any)
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

        # Generate a response
        with torch.no_grad():
            chat_history_ids = model.generate(
                bot_input_ids,
                max_length=1000,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,  # Lower temperature for more focused responses
                top_k=50,         # Consider top 50 tokens
                top_p=0.9         # Use nucleus sampling for diversity
            )

        # Decode the response
        response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

        # Print the model's response
        print(f"Kozaan: {response}")

# Your actual API key
api_key = 'a784c51d51cd44f3b7913cb5f7aca78e'  # Replace with your actual API key
# Start the chat
chat_with_model(api_key)

Chat with Kozaan! Type 'exit' to end the conversation.
Type 'news' to get the latest technology news.
You: Hi, what's up?
Kozaan: Not much, you?
You: not much, what's your name?
Kozaan: Oh, I don't know.
You: okay, i know you are good at conversing
Kozaan: Thanks, I try.
You: what's the latest news
Kozaan: Nothing new, what's up?
You: what is the latest news?
Kozaan: Nothing new, what's up?
You: what is the latest news
Kozaan: Here is the latest technology news:
Title: Nintendo Expands Switch Online's SNES Library With Four More Titles - Nintendo Life
Description: Arriving next week

Title: Microsoft Unveils DirectX Raytracing 1.2 With Huge Performance & Visual Improvements, Next-Gen Neural Rendering, Partnerships With NVIDIA, AMD & Intel - Wccftech
Description: Microsoft has announced DirectX Raytracing (DXR) 1.2 update alongside next-gen Neural Rendering support across NVIDIA, Intel & AMD hardware.

Title: 30 Hidden Gems On Amazon That You've Probably Never Heard Of - BuzzFeed
Descri

In [60]:
!nvidia-smi


Sat Mar 22 02:51:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             36W /   70W |   15044MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----